In [41]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import IBMQ, Aer, execute
import numpy as np

# IBMQ.load_account()

# Overview

We have to apply Grover's algorithm to 7 nodes, each encoded with a number from 00 to 11, and thus requiring two qubits. Note, however, that the graph's structure fixes the LSB of node 2 to "1", since it cannot be equal to A = 00 nor C = 10. This means that we have 13 input qubits to feed our Grover's algorithm.

The idea behind the oracle is to test all 19 remaining edges of the graph, each test representing whether the connected nodes are equal (outcome=0) or different (outcome=1). We then mark the corresponding state with a phase only if all tests are "1".

Each test is performed as described in function "two_group" further down below. The main point is that we cannot store all the tests into qubits for lack of space. Therefore, we do two tests at a time, and then do an AND between the outcomes: only if both tests were 1 is the overall result also 1, which we then store in a qubit.

The functions below are condensed versions of the basic operations needed throughout the circuit. They are optimized to condense adjacent single-qubit gates into single U3 gates.

In [29]:
#Optimized AND and OR
def AND(qc,a,b,out):
    qc.ry(np.pi/4,out)
    qc.cx(a,out)
    #qc.ry(-np.pi/4,out)
    qc.u3(3*np.pi/4,0,np.pi,out)
    #qc.h(out)
    qc.cx(b,out)
    #qc.h(out)
    qc.u3(3*np.pi/4,0,np.pi,out)
    #qc.ry(np.pi/4,out)
    qc.cx(a,out)
    qc.ry(-np.pi/4,out)
    
    
def OR(qc,a,b,out, safe=False):    #This flips the input states if safe=False, but leaves them unchanged if safe=TRUE
    qc.x(a)
    qc.x(b)
    
    qc.ry(np.pi/4,out)
    qc.cx(a,out)
    #qc.ry(-np.pi/4,out)
    qc.u3(3*np.pi/4,0,np.pi,out)
    #qc.h(out)
    qc.cx(b,out)
    #qc.h(out)
    qc.u3(3*np.pi/4,0,np.pi,out)
    #qc.ry(np.pi/4,out)
    qc.cx(a,out)
    
    #qc.ry(-np.pi/4,out)
    #qc.x(out)
    qc.u3(3*np.pi/4,-np.pi,0,out)
    
    if safe == True:
        qc.x(a)
        qc.x(b)
    
    
def iOR(qc,a,b,out,safe = False):     #Reverse of the OR function
    
    if safe == True:
        qc.x(b)
        qc.x(a)
    
    qc.u3(3*np.pi/4,np.pi,0,out)
    
    qc.cx(a,out)
    #qc.ry(-np.pi/4,out)
    qc.u3(3*np.pi/4,0,np.pi,out)
    #qc.h(out)
    qc.cx(b,out)
    #qc.h(out)
    qc.u3(3*np.pi/4,0,np.pi,out)
    #qc.ry(np.pi/4,out)
    qc.cx(a,out)
    
    qc.ry(-np.pi/4,out)
    
    qc.x(b)
    qc.x(a)
    
    
#Building blocks of a multi-controll-toffoli gate    

def block1(qc,a,b,target):      
    qc.cx(b,target)
    qc.u3(0,0,-np.pi/4,target)
    qc.cx(a,target)
    qc.u3(0,0,np.pi/4,target)
    qc.cx(b,target)
    qc.u3(np.pi/2,0,3*np.pi/4,target)
    
def block2(qc,a,b,target):
    qc.cx(b,target)
    qc.u3(0,0,-np.pi/4,target)
    qc.cx(a,target)
    qc.u3(0,0,np.pi/4,target)
    qc.cx(b,target)
    qc.u3(np.pi/2,np.pi/4,np.pi,b)
    qc.u3(np.pi/2,0,3*np.pi/4,target)
    
#--------------------------------------

#Optimized version of the mct, adapted for the average inversion part of Grover's algorithm (if avg_inv=True)
    
def mct(qc,ctrl,target,anc,avg_inv=False):
    nctrl=len(ctrl)
    nanc=len(anc)
    
    if avg_inv == False:
        qc.u3(np.pi/2,0,np.pi,target)
        
    qc.u3(np.pi/2,np.pi/4,np.pi,anc)
    
    block1(qc,ctrl[0],ctrl[1],anc[0])
    
    for i in range(1,nanc):
        block1(qc,ctrl[i+1],anc[i-1],anc[i])
        
    qc.cx(anc[-1],target)
    qc.u3(0,0,-np.pi/4,target)
    qc.cx(ctrl[-1],target)
    qc.u3(0,0,np.pi/4,target)
    qc.cx(anc[-1],target)
    qc.u3(0,0,-np.pi/4,target)
    qc.u3(0,0,np.pi/4,anc[-1])
    qc.cx(ctrl[-1],target)
    qc.cx(ctrl[-1],anc[-1])
    qc.u3(np.pi/2,np.pi*avg_inv,5*np.pi/4,target) #Changes if avg_inv = True
    qc.u3(0,0,np.pi/4,ctrl[-1])
    qc.u3(0,0,-np.pi/4,anc[-1])
    qc.cx(ctrl[-1],anc[-1])
    qc.u3(np.pi/2,np.pi/4,np.pi,anc[-1])
    
    for i in range(1,nanc)[::-1]:
        block2(qc,ctrl[i+1],anc[i-1],anc[i])
        
    qc.cx(ctrl[1],anc[0])
    qc.u3(0,0,-np.pi/4,anc[0])
    qc.cx(ctrl[0],anc[0])
    qc.u3(0,0,np.pi/4,anc[0])
    qc.cx(ctrl[1],anc[0])
    qc.u3(np.pi/2,0,3*np.pi/4,anc[0])
    
#Optimized multi-controlled-Z gate  
def mcz(qc,ctrl,target,anc):
    nctrl=len(ctrl)
    nanc=len(anc)
    
    #qc.u3(np.pi/2,0,np.pi,target)
        
    qc.u3(np.pi/2,np.pi/4,np.pi,anc)
    
    block1(qc,ctrl[0],ctrl[1],anc[0])
    
    for i in range(1,nanc):
        block1(qc,ctrl[i+1],anc[i-1],anc[i])
        
    qc.cx(anc[-1],target)
    qc.u3(0,0,-np.pi/4,target)
    qc.cx(ctrl[-1],target)
    qc.u3(0,0,np.pi/4,target)
    qc.cx(anc[-1],target)
    qc.u3(0,0,-np.pi/4,target)
    qc.u3(0,0,np.pi/4,anc[-1])
    qc.cx(ctrl[-1],target)
    qc.cx(ctrl[-1],anc[-1])
    qc.u3(0,0,np.pi/4,target)
    qc.u3(0,0,np.pi/4,ctrl[-1])
    qc.u3(0,0,-np.pi/4,anc[-1])
    qc.cx(ctrl[-1],anc[-1])
    qc.u3(np.pi/2,np.pi/4,np.pi,anc[-1])
    
    for i in range(1,nanc)[::-1]:
        block2(qc,ctrl[i+1],anc[i-1],anc[i])
        
    qc.cx(ctrl[1],anc[0])
    qc.u3(0,0,-np.pi/4,anc[0])
    qc.cx(ctrl[0],anc[0])
    qc.u3(0,0,np.pi/4,anc[0])
    qc.cx(ctrl[1],anc[0])
    qc.u3(np.pi/2,0,3*np.pi/4,anc[0])

In [30]:
#Average inversion part of Grover's algorithm: standard form, just simplified by merging neighbouring single-qubit gates
def average_inversion2(qc,q):
    idx = [i for i in range(13)]
    idx.remove(4)
    
  #  qc.barrier()
    
    qc.u3(np.pi/2,0,0,q[0:4])
    qc.u3(np.pi/2,0,0,q[5:14])
    
    mct(qc,[q[i] for i in idx], q[13], q[15:25],avg_inv=True)
    
    qc.u3(np.pi/2,np.pi,np.pi,q[0:4])
    qc.u3(np.pi/2,np.pi,np.pi,q[5:13])  

# Oracle

As stated above, we test two edges at a time and then store the result in a single qubit. In the following explanation node 1 is connected to node 2 and node 1 is connected to node 3. 

The test of the two edges is performed as follows:

1. A cnot from the lsb of node 1 to node 2
2. A cnot from the msb of node 1 to node 2
3. An OR between the lsb and msb of node 2 temporarily storing the result in what we call an operational ancilla.
4. A cnot from the lsb of node 3 to node 1
5. A cnot from the msb of node 3 to node 1
6. An OR between the lsb and msb of node 1 temporarily storing the result in another operational ancilla.
7. An AND between both operational ancillas storing the result in what we refer to as a storage quibit. This storage quibit will only be |1> when the two edges are valid

In [31]:
# Two-group function
def two_group(qc, base_node, t1, t2, ancilla, target, dirty=[]):
    """
    The two_group function is the basic element of the oracle. It performs two tests, first comparing the base_node with t1, 
    with stored results in t1, and then comparing base_node with t2, storing the results in base_node. 
    An OR is made between the qubits of the nodes that hold the results, returning whether they were equal or different to 
    the "ancilla" qubits. Then, an AND is performed between the ancillas, to store the combined result of both tests
    
    All these operations, except the AND, are then reversed, unless otherwise stated in the "dirty" list.
        
    Parameters
    ----------
        qc: QuantumCircuit
            Quantum circuit.
            
        base_node: list of 2 QuantumRegister  
            Origin node for three-group comparison. |lsb, msb>
            
        t1, t2: list of 2 QuantumRegister
            First, second target nodes for comparison |lsb, msb>
            
        ancilla: list of 2 QuantumRegister
            Two ancillas are needed for intermediate computations. |0> afterwards
            
        target: QuantumRegister
            Target quantum register for the result. 
            Target should be set to the ground state |0>.
            Target will change to |1> when all the target nodes are different from the base node
            but not necessarily different from each other.   
            
        dirty: list
            Flag which QuantumRegisters should not be reverted.
            Possible elements of the list are:
            'base_node', 't1', 'ancilla_1', 'ancilla_2'
            
    """
    possible_flags = ['base_node', 't1', 'ancilla_1', 'ancilla_2']
    for _ in dirty:
        if _ not in possible_flags:
            raise ValueError('Wrong flags!')
    
    qc.cx(base_node[0], t1[0])
    qc.cx(base_node[1], t1[1])
    
    qc.cx(t2[0], base_node[0])
    qc.cx(t2[1], base_node[1])
        
    qc.barrier()
    
    OR(qc, t1[0], t1[1], ancilla[0], safe = ('ancilla_1' in dirty))

    OR(qc, base_node[0], base_node[1], ancilla[1], safe = ('ancilla_2' in dirty) )
    
    qc.barrier()
    
    AND(qc,ancilla[0],ancilla[1],target)
    
    qc.barrier()
    
    if 'ancilla_1' not in dirty:
        iOR(qc, t1[0], t1[1], ancilla[0])

    if 'ancilla_2' not in dirty:
        iOR(qc, base_node[0], base_node[1], ancilla[1])
    
    qc.barrier()
    
    if 'base_node' not in dirty:
        qc.cx(t2[1], base_node[1])
        qc.cx(t2[0], base_node[0])
    
    if 't1' not in dirty:
        qc.cx(base_node[1], t1[1])
        qc.cx(base_node[0], t1[0])
        
        
# Two-group function
def two_group_inverse(qc, base_node, t1, t2, ancilla, target, dirty=[]):
    """
    This is just an inverted version of the two_group function, used to restore the target qubits back to their original
    state.
    
    
    Parameters
    ----------
        qc: QuantumCircuit
            Quantum circuit.
            
        base_node: list of 2 QuantumRegister  
            Origin node for three-group comparison. |lsb, msb>
            
        t1, t2: list of 2 QuantumRegister
            First, second target nodes for comparison |lsb, msb>
            
        ancilla: list of 2 QuantumRegister
            Two ancillas are needed for intermediate computations. |0> afterwards
            
        target: QuantumRegister
            Target quantum register for the result. 
            Target should be set to the ground state |0>.
            Target will change to |1> when all the target nodes are different from the base node
            but not necessarily different from each other.   
            
        dirty: list
            Flag which QuantumRegisters should not be reverted.
            Possible elements of the list are:
            'base_node', 't1', 'ancilla_1', 'ancilla_2'
            
    """
    possible_flags = ['base_node', 't1', 'ancilla_1', 'ancilla_2']
    for _ in dirty:
        if _ not in possible_flags:
            raise ValueError('Wrong flags!')
    
    if 't1' not in dirty:
        qc.cx(base_node[0], t1[0])
        qc.cx(base_node[1], t1[1])
    
    if 'base_node' not in dirty:
        qc.cx(t2[0], base_node[0])
        qc.cx(t2[1], base_node[1])
        
    qc.barrier()
    
    if 'ancilla_1' not in dirty:
        OR(qc, t1[0], t1[1], ancilla[0])

    if 'ancilla_2' not in dirty:
        OR(qc, base_node[0], base_node[1], ancilla[1])
    
    qc.barrier()
    
    AND(qc,ancilla[0],ancilla[1],target)
    
    qc.barrier()
    
    
    iOR(qc, t1[0], t1[1], ancilla[0],safe = ('ancilla_1' in dirty))

    
    iOR(qc, base_node[0], base_node[1], ancilla[1], safe = ('ancilla_2' in dirty))
    
    qc.barrier()
    
    
    qc.cx(t2[1], base_node[1])
    qc.cx(t2[0], base_node[0])
    

    qc.cx(base_node[1], t1[1])
    qc.cx(base_node[0], t1[0])
        
        

In [32]:
def full_oracle2(qc,q):
    '''
    Constructs the entire oracle. The tests in "edges2" are organized in such a way that the last time a node is tested, 
    the result of the comparison is left stored in its quibits, only being reversed after the phase has been applied.
    The last time the operational ancillas are used, they are also left "dirty" to spare operations.
    
    
    Qubit setup :
    
    q[0:4],q[5:14] : State qubits, from node 0 to 6
    q[4] : |1>
    q[14] : |0>
    q[15:17] : Operational ancillas
    q[17:26]: Storage ancillas
    q[26:] : Ancillas
    
    '''
    #Each tuple represents two tests: base_node-t1 and base_node-t2. Only one test is left after these.
    edges2 = [ # (base_node, t1, t2, dirty) 
    ('3','0','1', []),
    ('0','2','a', []),  #_________ last '0'
    ('1','0','b', ['t1']),
    ('4','1','b', ['t1']),         #_________ last '1'
    ('3','4','5', []),  #_________ last '4'
    ('6','4','d', ['t1']),
    ('5','6','d', []),  #_________ last '6'
    ('2','5','6', ['t1']),  #_________ last '2'
    ('3','6','2', ['t1','ancilla_1', 'ancilla_2'])   #_________ last '5'
    ] #Last one: 3-A ________________________ last '3'
    
    nodes = {
    'a': [q[14], q[14]],
    'b': [q[4], q[14]],
    'c': [q[14], q[4]],
    'd': [q[4], q[4]],
    '0': q[0:2],
    '1': q[2:4],
    '2': q[4:6],
    '3': q[6:8],
    '4': q[8:10],
    '5': q[10:12],
    '6': q[12:14],
        }
    
    op_ancillas= q[15:17]
    storage = q[17:26]
    ancillas = q[26:32]   
    
    for i, (base, t1_n, t2_n, dirty) in enumerate(edges2): #Runs through all the tests in edges2
        two_group(qc, nodes[base], nodes[t1_n], nodes[t2_n], op_ancillas, storage[i], dirty)
    
    #Last test (from node A to node 3)
    qc.cx(nodes['a'][0], nodes['3'][0])
    qc.cx(nodes['a'][1], nodes['3'][1])
    
    OR(qc, nodes['3'][0], nodes['3'][1], q[14])  # Use q[14] that holds a |0>
    
    
    qc.x(q[4]) # Clean q4 to use as ancilla
    
    #Flips the phase of a state if all qubits in storage, plus the last test's result, are "1"
    mcz(qc,[q[14]]+storage[:-1], storage[-1], ancillas+[q[4]])
    
    qc.x(q[4])
    
    
    iOR(qc, nodes['3'][0], nodes['3'][1], q[14])  # Revert the state of q[14]
    
    qc.cx(nodes['a'][1], nodes['3'][1])
    qc.cx(nodes['a'][0], nodes['3'][0])
    
    for i, (base, t1_n, t2_n, dirty) in enumerate(edges2[::-1]):  # Revert all the tests in edges2
        j = len(edges2)-i-1
        two_group_inverse(qc, nodes[base], nodes[t1_n], nodes[t2_n], op_ancillas, storage[j], dirty)

# Overview of the creative steps

Our strategy began with identifying that we would need to test the edges two by two to have enough quibits left to use as ancillas for the phase inversion mct. The most significant simplifications were:
1. Identifying that the lsb of node 2 had to be |1>
2. Wait until after the phase flip to revert some of the previous operations
3. Use the quibits that represented the fixed nodes (q[4] and q[14]) during the phase flip


# Constructing the full circuit

In [33]:
q = QuantumRegister(32)
c = ClassicalRegister(14)
qc = QuantumCircuit(q,c)

#State preparation

qc.h(q[0:4])
qc.h(q[5:14])
qc.x(q[4])


#Five complete iterations of Grover's algorithm
for _ in range(5):
    full_oracle2(qc,q)
    average_inversion2(qc,q)
    

#Measurement
# qc.measure(q[0:14],c[0:14])  # Our notation
# IMB Challenge notation. Convert our storage logic to the one required by the contest
qc.measure(q[0], c[1])
qc.measure(q[1], c[0])

qc.measure(q[2], c[3])
qc.measure(q[3], c[2])

qc.measure(q[4], c[5])
qc.measure(q[5], c[4])

qc.measure(q[6], c[7])
qc.measure(q[7], c[6])

qc.measure(q[8], c[9])
qc.measure(q[9], c[8])

qc.measure(q[10], c[11])
qc.measure(q[11], c[10])

qc.measure(q[12], c[13])
qc.measure(q[13], c[12])

In [34]:
from qiskit.transpiler.passes import Unroller
from qiskit.transpiler import PassManager

pass_ = Unroller(['u3', 'cx'])
pm = PassManager(pass_)
new_circuit = pm.run(qc) 
new_circuit.count_ops()

OrderedDict([('u3', 3399), ('cx', 2530), ('barrier', 360), ('measure', 14)])